In [37]:
import horovod.tensorflow as hvd

ModuleNotFoundError: No module named 'horovod'

In [1]:
import json
import tensorflow as tf2
import tensorflow.compat.v1 as tf
import numpy as np
from gpt2_keras.gpt2 import GPT2
from gpt2_keras.builder import original_gpt2
from gpt2_keras.builder.builder import build
# from .builder.builder import build
from gpt2_keras.encoder import get_encoder



def top_k_logits(logits, k):
    if k == 0:
        # no truncation
        return logits

    def _top_k():
        values, _ = tf.nn.top_k(logits, k=k)
        min_values = values[:, -1, tf.newaxis]
        return tf.compat.v1.where(
            logits < min_values,
            tf.ones_like(logits, dtype=logits.dtype) * -1e10,
            logits,
        )
    return tf.cond(
        pred=tf.equal(k, 0),
        true_fn=lambda: logits,
        false_fn=lambda: _top_k(),
    )




with open("./models/124M/hparams.json") as f:
    config = json.load(f)
#
gpt2 = GPT2(config, name='gpt2')

# x= tf.placeholder(dtype=tf.int32, shape=[None, None])
# y = gpt2(x)

# print(type(config))

# gpt2= build(config, "./models/124M/model.ckpt.data-00000-of-00001", name='gpt2')
gpt2= build(config, "./models/124M/model.ckpt", name='gpt2')

print(type(gpt2))
# print(gpt2.layers[1].layers) # The Transformer

embedding_layer = gpt2.layers[0]


print(embedding_layer)  # The Embedding Layer


print("printing vocab size:",  embedding_layer.vocab_size) #50257
print("printing word embedding:",  embedding_layer.word_embedding) #(50257 , 768)=


# gpt2.compile(
#     optimizer=tf2.optimizers.RMSprop(lr=0.01),
#     loss = tf2.keras.losses.MeanSquaredError(),
#     metrics = ['accuracy']
# )


print(gpt2.summary())
print("printing Transformer summary")
print(gpt2.layers[1].summary())

# print(gpt2.summary())
batch_size =1
max_seq_length = 1024
word_embedding = 768
tf.keras.backend.set_floatx('float64')
# input1 = np.random.randint(embedding_layer.vocab_size, size=(batch_size, 5, embedding_layer.word_embedding[-1]))



# input1 = np.random.randint(embedding_layer.vocab_size, size=(batch_size,max_seq_length))
# output = gpt2(input1)
# print(output)

model_dir = "./models/"
model_name = "124M"

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Restoring parameters from ./models/124M/model.ckpt
<class 'gpt2_keras.builder.gpt2.GPT2'>
printing vocab size: 50257
printing word embedding: <tf.Variable 'gpt2/embedding/word_embedding:0' shape=(50257, 768) dtype=float32, numpy=
array([[-0.11010301, -0.03926672,  0.03310751, ..., -0.1363697 ,
         0.01506208,  0.04531523],
       [ 0.04034033, -0.04861503,  0.04624869, ...,  0.08605453,
         0.00253983,  0.04318958],
       [-0.12746179,  0.04793796,  0.18410145, ...,  0.08991534,
        -0.12972379, -0.08785918],
       ...,
       [-0.04453601, -0.05483596,  0.01225674, ...,  0.10435229,
         0.09783269, -0.06952604],
       [ 0.1860082 ,  0.01665728,  0.04611587, ..., -0.09625227,
         0.07847701, -0.02245961],
       [ 0.05135201, -0.02768905,  0.0499369 , ...,  0.00704835,
         0.15519823,  0.12067825]], dtype=float32)>
Model: "gpt2"
______________________________

In [2]:
enc =get_encoder(model_name, model_dir)
raw_text = "What is interesting is the fact that the first"
raw_text1 = "My family is doing fine."
raw_text2 = "But, I think"
# raw_text += '<|endoftext|>'
bpe_tokens = enc.encode(raw_text)
bpe_tokens1 = enc.encode(raw_text1)
# bpe_tokens2 = enc.encode(raw_text2)



print("bpe_tokens: ", bpe_tokens)
print("bpe_tokens1: ", bpe_tokens1)

bpe_tokens:  [2061, 318, 3499, 318, 262, 1109, 326, 262, 717]
bpe_tokens1:  [3666, 1641, 318, 1804, 3734, 13]


In [3]:
# bpe_tokens1.append(50256)
# print(bpe_tokens1)
decoded = enc.decode(bpe_tokens)
print(decoded)

What is interesting is the fact that the first


In [4]:
while len(bpe_tokens1) != len(bpe_tokens):
    bpe_tokens1.append(220)
    
print("bpe_tokens1 AFTER \n pad: ", bpe_tokens1)

    

"""
The tokens have to be either padded or be of the same length to be input as batch.
"""

bpe_tokens1 AFTER 
 pad:  [3666, 1641, 318, 1804, 3734, 13, 220, 220, 220]


'\nThe tokens have to be either padded or be of the same length to be input as batch.\n'

In [5]:
#Without the endoftext : [3792, 534, 1641, 880, 30]

In [11]:
# start_token = enc.encoder['<|startoftext|>'] #
end_token = enc.encoder['<|endoftext|>']

# print(enc.decoder)

In [12]:
output2 = gpt2([bpe_tokens])
two_batch = [bpe_tokens, bpe_tokens1]
output3 = gpt2(two_batch)

print("**printing output2.shape**")
print(output2.shape)

print("printing argmax of logits")
output2_int = np.argmax(output2, axis=2)


print("**printing output3.shape**")
print(output3.shape)

print("printing output3")
print(output3)



print(output2_int)


**printing output2.shape**
(1, 9, 50257)
printing argmax of logits
**printing output3.shape**
(2, 9, 50257)
printing output3
tf.Tensor(
[[[ -37.217266  -36.886486  -40.356358 ...  -43.435127  -43.023254
    -37.09941 ]
  [ -84.74174   -82.64531   -88.28572  ...  -89.45007   -90.05545
    -84.677   ]
  [ -77.438644  -74.91486   -83.047714 ...  -85.73609   -89.438644
    -77.72409 ]
  ...
  [-103.48562  -103.50233  -107.460785 ... -105.617294 -109.159485
   -104.502495]
  [ -93.457886  -92.13289   -94.69766  ...  -90.95544   -96.78728
    -92.80166 ]
  [ -99.954926  -99.49396  -104.49782  ... -102.37962  -104.86124
   -101.482704]]

 [[ -33.07355   -32.334896  -35.237953 ...  -38.357666  -38.475765
    -33.09432 ]
  [-113.07895  -111.44274  -120.889984 ... -120.88999  -119.589806
   -115.100845]
  [-120.354065 -119.78439  -124.40173  ... -125.55103  -126.36237
   -122.76428 ]
  ...
  [ -73.21464   -69.45617   -74.36198  ...  -83.406815  -82.485275
    -72.76322 ]
  [ -32.82277   -29.0646

In [13]:
# from transformers import GPT2Tokenizer

# tokenizer = GPT2Tokenizer()


In [14]:
decoded_output = enc.decode(output2_int[0])
print(decoded_output)

"""
The decoded output is similar to that of the RNN where one token gives one output.
(This one output is the argmax of the gpt output, the immediate token that has the highest logit value)
(So, top_k is basically taking k argmaxes from one row vector of the logit matrix)
SO, for raw_text = "What is interesting is the fact that the first"

the output is "is the about that fact that the first time"

This means
<input>      <output>
What ==>     is
is ===>      the
interesting ===> about
is ===> that

"""

 is the about that fact that the first time


'\nThe decoded output is similar to that of the RNN where one token gives one output.\n(This one output is the argmax of the gpt output, the immediate token that has the highest logit value)\n(So, top_k is basically taking k argmaxes from one row vector of the logit matrix)\nSO, for raw_text = "What is interesting is the fact that the first"\n\nthe output is "is the about that fact that the first time"\n\nThis means\n<input>      <output>\nWhat ==>     is\nis ===>      the\ninteresting ===> about\nis ===> that\n\n'

In [15]:
encoded_space = enc.encode(" ")
print(encoded_space)

[220]


In [16]:
enc.decode([220])

' '

In [17]:
enc.decode([198])

'\n'

In [18]:
enc.decode([12])

'-'

In [19]:
# print("end_token is: ", start_token)
print("end_token is: ", end_token)

end_token is:  50256


In [20]:
print("printing output2 :", output2)
print("")

printing output2 : tf.Tensor(
[[[ -37.217266  -36.886486  -40.356358 ...  -43.435127  -43.023254
    -37.09941 ]
  [ -84.74174   -82.64531   -88.28572  ...  -89.45007   -90.05545
    -84.677   ]
  [ -77.438644  -74.91486   -83.047714 ...  -85.73609   -89.438644
    -77.72409 ]
  ...
  [-103.48562  -103.50233  -107.460785 ... -105.617294 -109.159485
   -104.502495]
  [ -93.457886  -92.13289   -94.69766  ...  -90.95544   -96.78728
    -92.80166 ]
  [ -99.95494   -99.49397  -104.49782  ... -102.37963  -104.86125
   -101.482704]]], shape=(1, 9, 50257), dtype=float32)



In [21]:
print("printing output3 :", output3)
print("printing decoded output3")
output3_int_arr = np.argmax(output3, axis=2)
decoded_output3 = [enc.decode(output3_int) for output3_int in output3_int_arr]
print(decoded_output3)

printing output3 : tf.Tensor(
[[[ -37.217266  -36.886486  -40.356358 ...  -43.435127  -43.023254
    -37.09941 ]
  [ -84.74174   -82.64531   -88.28572  ...  -89.45007   -90.05545
    -84.677   ]
  [ -77.438644  -74.91486   -83.047714 ...  -85.73609   -89.438644
    -77.72409 ]
  ...
  [-103.48562  -103.50233  -107.460785 ... -105.617294 -109.159485
   -104.502495]
  [ -93.457886  -92.13289   -94.69766  ...  -90.95544   -96.78728
    -92.80166 ]
  [ -99.954926  -99.49396  -104.49782  ... -102.37962  -104.86124
   -101.482704]]

 [[ -33.07355   -32.334896  -35.237953 ...  -38.357666  -38.475765
    -33.09432 ]
  [-113.07895  -111.44274  -120.889984 ... -120.88999  -119.589806
   -115.100845]
  [-120.354065 -119.78439  -124.40173  ... -125.55103  -126.36237
   -122.76428 ]
  ...
  [ -73.21464   -69.45617   -74.36198  ...  -83.406815  -82.485275
    -72.76322 ]
  [ -32.82277   -29.06462   -33.214638 ...  -42.720463  -42.29901
    -30.226212]
  [ -36.339565  -33.12221   -36.734196 ...  -45.

In [22]:
print(np.array(two_batch).shape)

(2, 9)


In [23]:
"""

For sparse_softmax_cross_entropy_with_logits, labels must have the shape [batch_size] and the dtype int32 or int64. Each label is an int in range [0, num_classes-1].
For softmax_cross_entropy_with_logits, labels must have the shape [batch_size, num_classes] and dtype float32 or float64.
Labels used in softmax_cross_entropy_with_logits are the one hot version of labels used in sparse_softmax_cross_entropy_with_logits.

Another tiny difference is that with sparse_softmax_cross_entropy_with_logits, you can give -1 as a label to have loss 0 on this label.

"""

'\n\nFor sparse_softmax_cross_entropy_with_logits, labels must have the shape [batch_size] and the dtype int32 or int64. Each label is an int in range [0, num_classes-1].\nFor softmax_cross_entropy_with_logits, labels must have the shape [batch_size, num_classes] and dtype float32 or float64.\nLabels used in softmax_cross_entropy_with_logits are the one hot version of labels used in sparse_softmax_cross_entropy_with_logits.\n\nAnother tiny difference is that with sparse_softmax_cross_entropy_with_logits, you can give -1 as a label to have loss 0 on this label.\n\n'

In [27]:
print(two_batch)

[[2061, 318, 3499, 318, 262, 1109, 326, 262, 717], [3666, 1641, 318, 1804, 3734, 13, 220, 220, 220]]


In [34]:
tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=np.array(two_batch)[:, 1:],
            logits=output3[:, :-1])

<tf.Tensor: id=15971, shape=(2, 8), dtype=float32, numpy=
array([[3.1975884 , 6.500888  , 1.1376996 , 2.762639  , 1.7081244 ,
        0.04474313, 1.8809332 , 4.913712  ],
       [6.4828143 , 1.9557304 , 5.2614756 , 2.9690232 , 1.339585  ,
        8.436424  , 6.941495  , 1.6103457 ]], dtype=float32)>

In [35]:
loss = tf.reduce_mean(
        input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=np.array(two_batch)[:, 1:],
            logits=output3[:, :-1])
)


In [36]:
print(loss)

tf.Tensor(3.5714514, shape=(), dtype=float32)


In [38]:
optimizer = tf.train.AdamOptimizer()

